In [1]:
import sys
import twitter
import json
import datetime
from datetime import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, ensemble
from sklearn.metrics import *
from sklearn.cross_validation import train_test_split, KFold
from autorizador import *


In [6]:
auth = get_credents()

In [5]:
def get_credents():
	creds = get_creds('secrets-manu.txt')
	auth=twitter.OAuth(creds.AccessToken,creds.AccessTokenSecret,creds.ConsumerKey, creds.ConsumerKeySecret)
	return auth

def save_tweet(tweet, f):
    # Replace HTML entities; function extracted from Borja Sotomayor Twitter Harvester 
    tweet['text'] = tweet['text'].replace("&gt;", ">").replace("&lt;", "<").replace("&amp;", "&")
    json.dump(tweet, f)
    f.write('\n')

def save_user_tweets(user, n, auth):
    t = twitter.Twitter(auth=auth)
    print("Fetching %i tweets from @%s" % (n, user))
    tweets = t.statuses.user_timeline(screen_name=user, count=n)
    print("  (actually fetched %i)" % len(tweets))
    for tweet in tweets:
        save_tweet(tweet, outfile)
        print(tweet['text'])

# read the filters file to track
# infile = open('filters_file.txt', 'r')
# track = ",".join(infile.read().strip().split("\n"))

def get_tweets(num_tweets, auth, filters_file, filter_words, filename_):
    '''
    Person Responsible: Manu Aragones

    + filter_file: list of words / phrases to be included in query IN FILE
    + filter_words: if filter_file not specified -> you can input filter manually
                    format: comma-separated list of phrases which will be used to
                            determine what Tweets will be delivered on the stream
    + num_tweets: the maximum number of tweets before break
    builds query for twitter API from user input

    Simplified fuction from Borja Sotomayor Twitter Harvester

    '''
    outf = open(filename_, "w")
    # Connect to the stream
    twitter_stream = twitter.TwitterStream(auth=auth)
    if filter_words is None and filters_file is None:
        stream = twitter_stream.statuses.sample()
    else:
        if filter_words is not None:
            track = ",".join(filter_words)
        elif filters_file is not None:
            infile = open(filters_file, 'r')
            track = ",".join(infile.read().strip().split("\n"))
        stream = twitter_stream.statuses.filter(track=track)
    # Fetch the tweets
    fetched = 0

    if num_tweets > 0:
        if outf != sys.stdout: print("Fetching %i tweets... " % num_tweets)
    else:
        signal.signal(signal.SIGINT, signal_handler)
        now = datetime.now().isoformat(sep=" ")
        msg = "[{}] Fetching tweets. Press Ctrl+C to stop.".format(now)
        if outf != sys.stdout: print(msg)

    for tweet in stream:
        # The public stream includes tweets, but also other messages, such
        # as deletion notices. We are only interested in the tweets.
        # See: https://dev.twitter.com/streaming/overview/messages-types
        if tweet.get('text'):
            # We also only want English tweets
            if tweet['lang'] == "en" or tweet['lang'] == "es":
                save_tweet(tweet, outf)
                fetched += 1
                if fetched % num_tweets == 0:
                    now = datetime.now().isoformat(sep=" ")
                    msg = "[{}] Fetched {:,} tweets.".format(now, fetched)
                    if outf != sys.stdout: 
                        print(msg)
                        stream.close()
                if num_tweets > 0 and fetched >= num_tweets:
                    stream.close()
                    break

In [7]:
get_tweets(20, auth, None, ['Trump','Hillary'], 'Trumpd2_test.txt')

Fetching 20 tweets... 
[2016-06-01 13:14:43.137111] Fetched 20 tweets.
